# Load & close an instance

In [ ]:
!lamin close

In [ ]:
from pathlib import Path
import pytest
import lamindb as ln
from lamindb.setup import settings
from lndb.dev._settings_store import instance_settings_file

## Load your own instance by name

If the user is the instance owner, just load the instance by name:

In [ ]:
ln.setup.load("mydata")  # CLI: lamin load mydata

In [ ]:
assert settings.instance.storage.is_cloud == False
assert settings.instance.name == "mydata"
assert (
    settings.instance.storage.root.as_posix() == Path("./mydata").resolve().as_posix()
)
assert settings.instance.storage.cache_dir is None
assert (
    settings.instance.db
    == f"sqlite:///{Path('./mydata').resolve().as_posix()}/mydata.lndb"
)

In [ ]:
# assume we move the storage location
!mv mydata mydata_new_loc
with pytest.raises(
    RuntimeError
):  # triggers because it does not find the SQLite file anymore
    ln.setup.load("mydata")
# now account for the new storage location
ln.setup.load("mydata", storage="./mydata_new_loc")
assert (
    settings.instance.storage.root.as_posix()
    == Path("./mydata_new_loc").resolve().as_posix()
)
assert settings.instance.storage.cache_dir is None
assert (
    settings.instance.db
    == f"sqlite:///{Path('./mydata_new_loc').resolve().as_posix()}/mydata.lndb"
)

In [ ]:
# another test case, this time with a manually configured instance name
ln.setup.load("pgtest-registered")
assert settings.instance.storage.is_cloud == True
assert settings.instance.name == "pgtest-registered"
assert settings.instance.storage.root_as_str == "s3://lndb-setup-ci"

This also works for remote instances:

In [ ]:
# ensure that the locally cached env file is deleted
instance_settings_file("lndb-setup-ci", "testuser1").unlink()

In [ ]:
ln.setup.load("lndb-setup-ci")  # lamin load lndb-setup-ci

In [ ]:
assert settings.instance.storage.is_cloud == True
assert settings.instance.storage.root_as_str == "s3://lndb-setup-ci"
assert (
    settings.instance._sqlite_file.as_posix() == "s3://lndb-setup-ci/lndb-setup-ci.lndb"
)

## Load an instance from another owner

If you have the permission, you can load an instance from another owner. 

In [ ]:
ln.setup.load("testuser1/mydata")  # lamin load "testuser1/mydata"

Load an instance from wrong owner.

In [ ]:
assert ln.setup.load("testuser2/mydata") == "instance-not-reachable"

## Close an instance

In [ ]:
ln.setup.close()

In [ ]:
# clean up
!lamin delete pgtest-registered
!docker stop pgtest-registered && docker rm pgtest-registered